In [1]:
from os import listdir
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [2]:
forge_base_path="TrainDutch/SigComp2009/Offline/Forged"
genuine_base_path="TrainDutch/SigComp2009/Offline/Genuine"

In [6]:
user=2
forged_image_filenames=listdir(forge_base_path+'/'+str(user))
genuine_image_filenames=listdir(genuine_base_path+'/'+str(user))

train_Xr=[]
train_Y=[]
for im in genuine_image_filenames:
    image_path=genuine_base_path+"/"+str(user)+'/'+im
    img=cv2.imread(image_path)
    train_Xr.append(img)
    train_Y.append(1)

  
        

for im in forged_image_filenames:
    image_path=forge_base_path+"/"+str(user)+'/'+im
    img=cv2.imread(image_path)
    train_Xr.append(img)
    train_Y.append(0) 
    
%matplotlib inline
train_X_br=[]
for i in train_Xr:
    train_X_br.append(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY))
print(np.shape(train_X_br[0])) 
#plt.imshow(train_X_br[0])

dim=(120,120)
train_X=[]
for i in train_X_br:
     train_X.append(cv2.resize(i, dim, interpolation = cv2.INTER_AREA))
#plt.imshow(train_X[0])
#plt.imshow(train_X_br[0])


train_X=np.array(train_X)
train_X = train_X.reshape(-1, 120,120, 1)
#test_X = test_X.reshape(-1, 120,120, 1)
print(np.shape(train_X[0]))
train_X = train_X.astype('float32')
#test_X = test_X.astype('float32')
train_X = train_X / 255
#test_X = test_X / 255


train_Y_one_hot = to_categorical(train_Y)
#test_Y_one_hot = to_categorical(test_Y)
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

batch_size = 75
epochs = 30
num_classes = 2


signature_model = Sequential()
signature_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(120,120,1),padding='same'))
signature_model.add(LeakyReLU(alpha=0.01))
signature_model.add(MaxPooling2D((2, 2),padding='same'))
signature_model.add(Dropout(0.25))

signature_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
signature_model.add(LeakyReLU(alpha=0.01))
signature_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
signature_model.add(Dropout(0.25))

signature_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
signature_model.add(LeakyReLU(alpha=0.01))                  
signature_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
signature_model.add(Dropout(0.4))

signature_model.add(Flatten())
signature_model.add(Dense(128, activation='linear'))
signature_model.add(LeakyReLU(alpha=0.01))   
signature_model.add(Dropout(0.3))
signature_model.add(Dense(num_classes, activation='softmax'))


signature_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])
#signature_model.summary()


signature_train = signature_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))


model_json = signature_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
signature_model.save_weights("model.h5")
print("Saved model to disk")

(862, 1947)
(120, 120, 1)
Train on 124 samples, validate on 31 samples
Epoch 1/30
124/124 [==============================] - 9s 70ms/step - loss: 0.5345 - acc: 0.5484 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/30
124/124 [==============================] - 8s 66ms/step - loss: 0.6499 - acc: 0.9597 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/30
124/124 [==============================] - 7s 60ms/step - loss: 0.6499 - acc: 0.9597 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/30
124/124 [==============================] - 7s 59ms/step - loss: 0.6499 - acc: 0.9597 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/30
124/124 [==============================] - 7s 59ms/step - loss: 0.6499 - acc: 0.9597 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/30
124/124 [==============================] - 9s 72ms/step - loss: 0.6499 - acc: 0.9597 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/30
124/124 [==============================] - 7s 59ms/step - loss: 0.6499 - acc: 0.9597 - val_loss:

In [5]:
signature_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 120, 120, 32)      320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 120, 120, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
__________

In [9]:
pred=signature_model.predict(valid_X,batch_size=32)

In [10]:
pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [11]:
valid_label

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)